In [1]:
import torch
import torch.nn as nn
import numpy as np
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
from torchvision.models import efficientnet_b1, EfficientNet_B1_Weights
import os

IMG_SIZE = 224
NUM_CLASSES = 3
CKPT_PATH = "models/park.pth"
ONNX_PATH = "models/park.onnx"
DEVICE = "cpu"


def build_model(num_classes=NUM_CLASSES, pretrained=True):
    # weights = MobileNet_V3_Large_Weights.IMAGENET1K_V1 if pretrained else None
    # model = mobilenet_v3_large(weights=weights)
    weights = MobileNet_V3_Small_Weights.IMAGENET1K_V1 if pretrained else None
    model = mobilenet_v3_small(weights=weights)
    if hasattr(model, 'classifier'):
        in_features = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(in_features, num_classes)
    return model

def build_model1(num_classes=NUM_CLASSES, pretrained=True):
    weights = EfficientNet_V2_S_Weights.IMAGENET1K_V1 if pretrained else None
    model = efficientnet_v2_s(weights=weights)
    
    in_features = model.classifier[-1].in_features
    model.classifier[-1] = nn.Linear(in_features, num_classes)
    
    return model

model = build_model(num_classes=NUM_CLASSES, pretrained=False)
checkpoint = torch.load(CKPT_PATH, map_location=DEVICE)
model.load_state_dict(checkpoint["model_state"])
model.eval()


dummy_input = torch.randn(1, 3, IMG_SIZE, IMG_SIZE, device=DEVICE)

input_names = ["input"]
output_names = ["output"]

torch.onnx.export(
    model,
    dummy_input,
    ONNX_PATH,
    export_params=True,
    opset_version=11, 
    do_constant_folding=True,
    input_names=input_names,
    output_names=output_names,
    dynamic_axes={ "input": {0: "batch_size"}, "output": {0: "batch_size"} }
)

In [2]:
import onnx
onnx_model = onnx.load("models/park.onnx")
onnx.checker.check_model(onnx_model)

In [4]:
import onnx
from onnx import version_converter

# После quantize_static добавьте:
# Загружаем квантованную модель
model = onnx.load("models/efficientnetv2s.onnx")

# Конвертируем в opset 13 (хорошо поддерживается)
converted_model = version_converter.convert_version(model, 17)

# Сохраняем совместимую модель
compatible_model = "models/efficientnetv2s_int8_2.onnx"
onnx.save(converted_model, compatible_model)
onnx.checker.check_model(onnx.load(compatible_model))

print(f"✓ Model converted to opset 13: {compatible_model}")

✓ Model converted to opset 13: models/efficientnetv2s_int8_2.onnx
